<a href="https://colab.research.google.com/github/smohanna/EAEE4000_Rainfall_Intensity/blob/main/01_historico_chirps_RS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q xee rioxarray earthengine-api geemap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.9 MB/s eta 0:00:00


In [5]:
import ee
import xarray as xr
import xee   # registra o engine 'ee'
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

ee.Authenticate()
ee.Initialize(project='pacific-song-279418')  # seu projeto


Mounted at /content/drive


In [6]:
# variável e período
var_name   = 'precipitation'         # banda do CHIRPS
time_start = '2000-01-01'
time_end   = '2024-06-01'

# bounding box aproximado do RS (lonMin, latMin, lonMax, latMax)
xMin, yMin, xMax, yMax = -57.0, -34.0, -49.0, -27.0

# coleção CHIRPS diária
collection = (ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
              .filterDate(time_start, time_end)
              .select(var_name))

print("Número de imagens:", collection.size().getInfo())


Número de imagens: 8918


In [7]:
print(">>> Baixando dados para Xarray (pode levar alguns minutos)...")

ds = xr.open_dataset(
    collection,
    engine='ee',
    scale=0.1,
    projection=collection.first().select(0).projection(),
    geometry=ee.Geometry.Rectangle(xMin, yMin, xMax, yMax),
).rename({'lon': 'x', 'lat': 'y'})

ds = ds.transpose('time', 'y', 'x')

print("Datas:", ds[var_name].time.min().values, "→", ds[var_name].time.max().values)
print("Shape:", ds[var_name].shape)
print(ds)


>>> Baixando dados para Xarray (pode levar alguns minutos)...
Datas: 2000-01-01T00:00:00.000000000 → 2024-05-31T00:00:00.000000000
Shape: (8918, 141, 160)
<xarray.Dataset> Size: 805MB
Dimensions:        (time: 8918, x: 160, y: 141)
Coordinates:
  * time           (time) datetime64[ns] 71kB 2000-01-01 ... 2024-05-31
  * x              (x) float64 1kB -56.98 -56.92 -56.88 ... -49.12 -49.08 -49.02
  * y              (y) float64 1kB -27.02 -27.07 -27.12 ... -33.92 -33.98 -34.02
Data variables:
    precipitation  (time, y, x) float32 805MB ...
Attributes: (12/20)
    date_range:               [347155200000, 1646006400000]
    description:              <p>Climate Hazards Group InfraRed Precipitation...
    keywords:                 ['chg', 'climate', 'geophysical', 'precipitatio...
    period:                   1
    period_mapping:           [347155200000, 1646006400000]
    product_tags:             ['precipitation', 'climate', 'weather', 'geophy...
    ...                       ...
    vi

In [8]:
df_hist = ds[var_name].to_dataframe().reset_index()

df_hist = df_hist.rename(columns={
    'time': 'date',
    var_name: 'rain_mm'
})

# tira NaNs (oceano/bordas)
df_hist = df_hist.dropna(subset=['rain_mm'])

# colunas de tempo
df_hist['year']  = pd.to_datetime(df_hist['date']).dt.year
df_hist['month'] = pd.to_datetime(df_hist['date']).dt.month
df_hist['day']   = pd.to_datetime(df_hist['date']).dt.dayofyear

df_hist.head(), df_hist.shape


(        date       y       x  rain_mm  year  month  day
 0 2000-01-01 -27.025 -56.975      0.0  2000      1    1
 1 2000-01-01 -27.025 -56.925      0.0  2000      1    1
 2 2000-01-01 -27.025 -56.875      0.0  2000      1    1
 3 2000-01-01 -27.025 -56.825      0.0  2000      1    1
 4 2000-01-01 -27.025 -56.775      0.0  2000      1    1,
 (158677974, 7))

In [9]:
caminho_hist = "/content/drive/MyDrive/chuva_RS_chirps_2000_2024.csv"
df_hist.to_csv(caminho_hist, index=False)
print("Histórico salvo em:", caminho_hist)


Histórico salvo em: /content/drive/MyDrive/chuva_RS_chirps_2000_2024.csv
